In [1]:
import base64
from io import BytesIO
from PIL import Image

def base64_encode_pil_image(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def base64_decode_pil_image(base64_img):
    base64_img_bytes = base64_img.encode('utf-8')
    decoded_image_data = base64.decodebytes(base64_img_bytes)
    return Image.open(BytesIO(decoded_image_data))

#img_str = base64_encode_pil_image(image)
#im = base64_decode_pil_image(img_str)

In [3]:
from kafka import KafkaProducer
from datetime import datetime
from uuid import uuid4
import json 
import ast

def get_id():
    return  datetime.now().strftime('%Y%m-%d%H-%M%S-') + str(uuid4())

def create_status_message(id=get_id(),name=None,status=None,type=None,duration=None,topic=None):  
    d1 = { key:item for key,item in locals().items() if item is not None} 
    d2 = {'date' : datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    return {**d1, **d2}

def create_question_message(question=None,session_id=''):
    return {'session_id':session_id ,'id':get_id(), 'instruction':question}
    
def create_answer_message (answer=None,id=None,time=None):
    return { key:item for key,item in locals().items() if item is not None}
    
def dict_to_byte_str(data):    
    return json.dumps(data).encode('utf-8')    
    
def send_byte_str_to_topic(byte_str,topic):    
    producer = KafkaProducer(bootstrap_servers='ml-integration.research.dev.seeburger.de:9092',compression_type='gzip')
    producer.send(topic, byte_str)
    producer.flush()      
    
def send_to_topic(data,topic):    
    return send_byte_str_to_topic(dict_to_byte_str(data),topic)
    
#byte_str = dict_to_byte_str(create_question_message(question='what is an API'))   
#send_to_topic(byte_str,'test-topic')
#create_answer_message(id='fsfkjhlk') 

## for qr 

In [ ]:
def wait_for_response_image(consumer,topic=None,key='answer',id=None):
    while True:
        message = consumer.poll(120.0)
        if not message or  message.error():
            continue
                
        dict_str = message.value().decode('utf-8')
        value = ast.literal_eval(dict_str)
        print('wait_for_response_image',id)
        if (value['id'] == id):
            if key in value.keys():
                image = base64_decode_pil_image(value['answer'])
                return(image)
            
def send_instruction(input,topic=None,session_id=''):
    #topic = 'qr-01'
    question = create_question_message(question=str(input),session_id=session_id)
    send_to_topic(question,topic)
    return question['id']            

In [2]:
import time
from confluent_kafka import Consumer
import time
import ast
from confluent_kafka import Consumer, TopicPartition

def get_last_n_messages(n,topic=None,group_id=None):
    
    consumer = Consumer({
        'bootstrap.servers': 'ml-integration.research.dev.seeburger.de:9092',
        'group.id': group_id,
        'auto.offset.reset': 'earliest'
    })
    consumer.subscribe([topic])
    topic_name=topic
    offsets = consumer.get_watermark_offsets(TopicPartition(topic_name, 0))
    #print(offsets)
    partition = TopicPartition(topic_name, 0, offsets[1] - n )
    consumer.assign([partition])
    consumer.seek(partition)

    result = []
    running = True
    while running:
        try: 
            #print('start')
            message = consumer.poll(1.0)

            if not message:
                print('none')
                running = False
                break
                #time.sleep(120) # Sleep for 2 minutes

            if message.error():
                print(f"Consumer error: {message.error()}")
                continue

            #print(message)    
            #print(f"Received message: {message.value().decode('utf-8')}")
            result.append(ast.literal_eval(message.value().decode('utf-8')))
            #print(f"Received message: {message.offset()}")
        except:
            # Handle any exception here
            pass
    return result    
    #    finally:
    #        consumer.close()
    #        pass
    #consumer.close()

In [5]:
import time
import ast
from confluent_kafka import Consumer, TopicPartition

def get_last_message(group,topic):
    consumer = Consumer({
    'bootstrap.servers': 'ml-integration.research.dev.seeburger.de:9092',
    'group.id': group,
    'auto.offset.reset': 'earliest'
    })
    consumer.subscribe([topic])  
    
    offsets = consumer.get_watermark_offsets(TopicPartition(topic, 0))
    #print(offsets)
    partition = TopicPartition(topic, 0, offsets[1] - 1)
    consumer.assign([partition])
    consumer.seek(partition)
    message = consumer.poll(1.0)
    dict_str = message.value().decode('utf-8')
    try:
        return ast.literal_eval(dict_str)
    except Exception as e:
        return dict_str
    
#get_last_message('my-consumer-12','test-topic')

In [1]:
import time
import ast
from confluent_kafka import Consumer, TopicPartition

def create_and_subscribe_consumer(group = None,topic = None):
    consumer = Consumer({
        'bootstrap.servers': 'ml-integration.research.dev.seeburger.de:9092',
        'group.id': group,
        'auto.offset.reset': 'latest'
        })
    consumer.subscribe([topic])  
    return consumer

def get_next_message(consumer,id=None):
    running = True
    while running:
        try: 
            #print('start')
            message = consumer.poll(120.0)

            if not message:
                continue
                #time.sleep(120) # Sleep for 2 minutes

            if message.error():
                print(f"Consumer error: {message.error()}")
                continue
                
                
            if id is None:    
                return(message)
            else:
                dict_str = message.value().decode('utf-8')
                value = ast.literal_eval(dict_str)
                if (value['id'] == id):
                    return(message)

            #running = False
            #break
        except:
            # Handle any exception here
            pass